In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import cv2
from PIL import Image

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Version 3- Added Anamoly ids  and Area computation for Bounding boxes

In [ ]:
import matplotlib.pyplot as plt
import pydicom
import pydicom.data  # we need pydicom to read all the dicom files some thing new for beginners

In [ ]:
base='../input/siim-covid19-detection/train/'
#../input/siim-covid19-detection/train/00086460a852/9e8302230c91/65761e66de9f.dcm

In [ ]:
file='65761e66de9f.dcm'
#filename = pydicom.data.data_manager.get_files(base,  '00')[0]
#print(file_name)  
ds = pydicom.dcmread('../input/siim-covid19-detection/train/00086460a852/9e8302230c91/65761e66de9f.dcm')
  
plt.imshow(ds.pixel_array, cmap=plt.cm.bone)  # set the color map to bone
plt.show()

In [ ]:
import fastai
import fastai.medical
from fastai.basics import *
from fastai.vision.all import *
from fastai.medical.imaging import *

import pydicom

import pandas as pd

In [ ]:
import glob
# Lets check the meta data for pydicom
#items=get_dicom_files(base)
#print(items)
#df_trn = pd.DataFrame.from_dicoms(items,px_summ=False)
'''
col_list=[ 'StudyID','PatientID','ImageType', 'StudyInstanceUID', 'SeriesInstanceUID',
        'SOPClassUID', 'SOPInstanceUID', 'PhotometricInterpretation', 'Rows', 'Columns', 'BitsAllocated',
       'BitsStored', 'HighBit', 'PixelRepresentation', 'fname','BodyPartExamined',]

df_trn[[ 'StudyID','PatientID','ImageType', 'StudyInstanceUID', 'SeriesInstanceUID',
        'SOPClassUID', 'SOPInstanceUID', 'PhotometricInterpretation', 'Rows', 'Columns', 'BitsAllocated',
       'BitsStored', 'HighBit', 'PixelRepresentation', 'fname','BodyPartExamined',]]
'''

In [ ]:
#df_trn.to_csv('df_trn_dicom.csv',index=False)

In [ ]:
#df_trn.values[10]

In [ ]:
#df_trn.StudyID.nunique()

In [ ]:
#Mapping directory structure to Ids in Dicom
#df_trn[df_trn.SeriesInstanceUID=='9e8302230c91'][col_list] #series instance uid is sub dir,studyinstanceuid is upper directory,SOPinstanceUID is image id

In [ ]:
#items[0]

In [ ]:
import pandas as pd
train_image=pd.read_csv('../input/siim-covid19-detection/train_image_level.csv')
train_image[train_image.StudyInstanceUID=='00292f8c37bd'].label.values #Non Pneumonia label look like this

*Every Covid 19 is considered Pneuomonia  but every pneumonia is not a Covid
So if Pneumonia is 0  it means there cannot be covid . If Pneuomonia  is 1 it means there can be various kinds of Covid 19 Pneumonia patterns such as Typical /Atypical or Indertiminate *
[https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7297525/] )

In [ ]:
!cp ../input/basic-exploration-eda-duplicate-nonduplicate/df_trn_dicom.csv /kaggle/working/

In [ ]:
# All values are 0 if its  Not a penuomonia
train_study=pd.read_csv('../input/siim-covid19-detection/train_study_level.csv')
train_study['StudyInstanceUID']=train_study.id.apply(lambda x: x.split('_')[0])


train_study[train_study["Negative for Pneumonia"]==1].sum(0)

In [ ]:
''' 
OpacityCount = train_image['label'].str.count('opacity')
#np.where(OpacityCount==0)[0].shape
#train_image[train_image.boxes.isnull()]#.StudyInstanceUID.nunique()
#set(train_image[train_image.boxes.isnull()].StudyInstanceUID.unique())-set(count_df_train_img[count_df_train_img.boxescnt==0].StudyInstanceUID.values)
train_image['opacity_count']=OpacityCount
np.where(OpacityCount==0)[0].shape,train_image[train_image.boxes.isnull()].StudyInstanceUID.unique().shape
 train_image[(train_image['opacity_count']==0) ].StudyInstanceUID.nunique() 
train_image['anamoly_flag']=train_image.apply(lambda x:1 if  (x['opacity_count']==0 and  x['Negative for Pneumonia']  !=0)  or (x['opacity_count']==0 and  x['Negative for Pneumonia']  !=1) 
                                              else 0,axis=1 ) 
'''

In [ ]:
#Computing  the presence of BBx there or not  and areas of various bboxes
import ast,numpy as np
train_image.boxes.fillna('',inplace=True)

train_image['bbx_cnt']=train_image.boxes.apply(lambda x: len(ast.literal_eval(x) ) if  x !=''  else 0)
train_image['area']=train_image.boxes.apply(lambda x : float(np.sum([ a['width']*a['height'] for a in 
                                                               ast.literal_eval(x)])) if  x !='' else 0)

In [ ]:
plt.hist(train_image['area'])  #total area for all the bounding boxes for Xray

In [ ]:
#Getting anamoly ids the ones which are Covid positive but no BBX but other way round is 0 so we are good
anamoly_ids=pd.merge(train_image[train_image.bbx_cnt==0],train_study[train_study["Negative for Pneumonia"]==0],on='StudyInstanceUID').id_x.values.tolist()
print('Total No of Anamoly ids ',len(anamoly_ids))
train_image[~train_image.id.isin(anamoly_ids)].shape
#train_image.shape

In [ ]:
#train_image=pd.merge(train_image, train_study, on='StudyInstanceUID')
#train_image.index.size

In [ ]:
#train_image['anamoly_flag']=train_image.apply(lambda x:1 if  (x['opacity_count']==0 and  train_study[train_study.StudyInstanceUID==x['StudyInstanceUID'] ]['Negative for Pneumonia'].values[0] !=0) else 0,axis=1 ) 

In [ ]:
# All values are 0 if its  Not a penuomonia
train_study=pd.read_csv('../input/siim-covid19-detection/train_study_level.csv')
train_study[train_study["Negative for Pneumonia"]==1].sum(0)

Looks like Atypical patterns goes by its meaning interms of its occurances  in Images , and Typical also same . However some thing to notics is that there are around 1k appearance those are categorized as  Indeterminate. 

In [ ]:
#So all patterns that are typical have No Indeterminate and Atypical patterns
train_study[(train_study["Negative for Pneumonia"]==0) & 
            ((train_study["Typical Appearance"]==1))].sum(0)

In [ ]:
#Same goes for Indeterminate
train_study[(train_study["Negative for Pneumonia"]==0) & 
            ((train_study["Indeterminate Appearance"]==1))].sum(0)

In [ ]:
#Same goes for Atypical
train_study[(train_study["Negative for Pneumonia"]==0) & 
            ((train_study["Atypical Appearance"]==1))].sum(0)

Conclusion is This Problem Is multiclass objection Detection  Not Multilabel Multiclass 

In [ ]:
train_image.StudyInstanceUID.count() ,train_study.id.count() 
#more rows than train study

In [ ]:
train_study.head()

In [ ]:
count_df_train_img

In [ ]:
#checking if a given StudyUID can have more than one DCMs
count_df_train_img=train_image.iloc[:,:-2].groupby('StudyInstanceUID').count().reset_index()
count_df_train_img.columns=['StudyInstanceUID','idcnt','boxescnt','labelcnt' ]
count_df_train_img

In [ ]:
#Lets check one  
train_image[train_image.StudyInstanceUID=='ffcb4630f46f']

One sop instance UID that is upper most dir will have multiple Series Instance UID which could possibly mean multiple x ray reports one may have infection other instance may not have an infection..negative report

In [ ]:
!ls -l ../input/siim-covid19-detection/train/ffcb4630f46f/

In [ ]:
!ls -l ../input/siim-covid19-detection/train/ffcb4630f46f/4a4a9ae34a66

In [ ]:
 count_df_train_img.shape  

Loooking ids greater than 2 count

In [ ]:
count_df_train_img[count_df_train_img.idcnt>1  ].head()

In [ ]:
#Looking at ones having boxes greater than 1 real duplicates with no other labels 

train_image[train_image.StudyInstanceUID=='7416b5cbc531']

In [ ]:


count_df_train_img[(count_df_train_img.idcnt==1) ] 

In [ ]:
count_df_train_unique= count_df_train_img[(count_df_train_img.idcnt==1) ] 

In [ ]:
train_image.head()

In [ ]:
#merge with train img id having 
train_image_unique=pd.merge(count_df_train_unique,train_image,on='StudyInstanceUID')
train_image_unique.head()

In [ ]:
train_image_unique[train_image_unique.boxes.isnull() ].shape # all Nan have no boxes numbering 1705

In [ ]:
#validating with train study
train_study['StudyInstanceUID']=train_study.id.apply(lambda x: x.split('_')[0])

 

In [ ]:
train_study_unique=pd.merge(count_df_train_unique,train_study,on='StudyInstanceUID')
train_study_unique.head()

In [ ]:
train_study_unique[train_study_unique.boxescnt==0 ]["Negative for Pneumonia"].value_counts()
# oh so there are  82 odd instances where are there no bbox provided even though patients have pneumonia

In [ ]:
#Further classifying them
train_study_unique[train_study_unique.boxescnt==0 ].sum(0)
#mostly are atypical  
#open question from domain expert why should that be

In [ ]:
train_study.sum(0)

In [ ]:
#lets check regular Atypical cases 

train_study_unique[train_study_unique.boxescnt!=0 ].sum(0)
#ok that is fine 386 out of 474 have no irregularity

That is it very  Simple EDA . Could help in building a good CV .Please post your opinions or any questions
